In [71]:
	
%reload_ext sql
%reload_ext mocodo

In [72]:
	
%reload_ext sql
%reload_ext mocodo

In [73]:
%config SqlMagic.dsn_filename = "connexion_DB.ini" 
%sql --section pg
%config SqlMagic.displaylimit = 0 #pas de limite dans le nombre de tuples du résultat

Connecting to 'pg'

In [74]:
%%sql
-- 

begin;

drop schema if exists optimisation cascade;
create schema optimisation;
set search_path to optimisation ;

--drop table if exists clients cascade;
create unlogged table  clients(
	NumC 		integer primary key,
	NomC		varchar(35),
	AdresseC	text
);

--drop table if exists produits cascade;
	create unlogged table produits(
	NumP		integer primary key,
	NomP		varchar(35),
	descriptif	text
);

--drop table if exists commandes cascade;
create unlogged table commandes(
	DateCom		date,
	NumC		integer ,
	Commentaire	text,
	primary key(DateCom,NumC)
);

--drop table if exists livraisons;
create unlogged table livraisons(
	DateLiv		date,
	DateCom		date,
	NumC		Integer,
	Prestataire	varchar,
	primary key(DateLiv,DateCom,NumC)
);

--drop table if exists Concerne;
create unlogged table concerne(
	NumP		integer,
	DateCom 	date,
	NumC		integer,
	Quantite	integer,
	primary key (NumP,DateCom,NumC)
);

	commit;

	begin;
-------------------------------
	
-- Procédure qui crée :
----- nbtuples clients
----- nbtuples produits
----- jusqu'à 4 commandes par clients  
----- jusqu'à 6 produits par commande ; donc table concerne en moyenne 12*nbtuples
drop procedure if exists rempli_tables;
create procedure rempli_tables(nbtuples integer)
as $$
declare
k			integer;
un_numP		integer;
compt		integer;
une_date	date;
curs_com 	cursor for select DateCom, numc 
				from commandes;
begin


	INSERT INTO clients (numc , nomc , adressec) 
	SELECT g, 'nomc_'||((random() * nbtuples * 4 / 5)::integer)::varchar, md5(g::text)||md5((2*g)::text)||md5((3*g)::text)  
	FROM generate_series(0,nbtuples-1) as g;

	INSERT INTO produits (nump , nomp , descriptif) 
	SELECT g, 'nomp_'||((random() * nbtuples * 4 / 5)::integer)::varchar, md5(g::text)||md5((2*g)::text)||md5((3*g)::text)  
	FROM generate_series(0,nbtuples-1) as g;
	

	for g in 1..4*(nbtuples-1) loop
		une_date = CURRENT_TIMESTAMP - ((random() * 3650)::integer||'day')::interval;
		un_nump=(random()*(nbtuples-1))::integer;
		insert into commandes (DateCom, numc, commentaire) 
			select 	une_date, un_nump, md5(g::text)||md5((2*g)::text)||md5((3*g)::text) on conflict do nothing;
	end loop;
	
	
		for t in curs_com 
		loop
			k=(random() *6)::integer;
			for i in 1..k loop
				un_numP = (random()*(nbtuples-1))::integer;
				insert into concerne (NumP,datecom,numc,quantite)
				values (un_nump,t.Datecom,t.numc,(random()*100)::integer) on conflict do nothing;
				if i>3 then
					une_date = t.datecom+((random() * 30)::integer||'days')::interval;
					insert into livraisons (DateLiv, DateCom, numC, prestataire)
					values(une_date,t.datecom,t.numc,'prest'||((random() *20)::integer)::varchar) on conflict do nothing;
				end if;
			end loop;
		end loop; 


end;

$$ language plpgsql;

do $$
begin
	call optimisation.rempli_tables(50000);
end
$$;


commit;


begin;
alter table commandes add CONSTRAINT fk_commande_client foreign key (numc) references clients(NumC) on delete cascade not valid;
alter table livraisons add constraint fk_livraison_commande foreign key (DateCom,NumC) references commandes(DateCom,NumC) on delete cascade not valid;
alter table concerne add constraint fk_concerne_commande foreign key (DateCom,NumC) references commandes(DateCom,NumC) on delete cascade not valid;
alter table concerne add constraint fk_concerne_produit	foreign key (NumP) references produits(NumP) not valid;

commit;


Running query in 'pg'

++
||
++
++

In [75]:
%%sql
analyse optimisation.clients, optimisation.commandes, optimisation.produits, optimisation.concerne,optimisation.livraisons;

Running query in 'pg'

++
||
++
++

In [ ]:
%%sql
SELECT * FROM produits

In [76]:
%%sql
SELECT * FROM produits LIMIT 10

Running query in 'pg'

10 rows affected.

nump,nomp,descriptif
0,nomp_17051,cfcd208495d565ef66e7dff9f98764dacfcd208495d565ef66e7dff9f98764dacfcd208495d565ef66e7dff9f98764da
1,nomp_251,c4ca4238a0b923820dcc509a6f75849bc81e728d9d4c2f636f067f89cc14862ceccbc87e4b5ce2fe28308fd9f2a7baf3
2,nomp_849,c81e728d9d4c2f636f067f89cc14862ca87ff679a2f3e71d9181a67b7542122c1679091c5a880faf6fb5e6087eb1b2dc
3,nomp_38483,eccbc87e4b5ce2fe28308fd9f2a7baf31679091c5a880faf6fb5e6087eb1b2dc45c48cce2e2d7fbdea1afc51c7c6ad26
4,nomp_38952,a87ff679a2f3e71d9181a67b7542122cc9f0f895fb98ab9159f51fd0297e236dc20ad4d76fe97759aa27a0c99bff6710
5,nomp_30319,e4da3b7fbbce2345d7772b0674a318d5d3d9446802a44259755d38e6d163e8209bf31c7ff062936a96d3c8bd1f8f2ff3
6,nomp_33121,1679091c5a880faf6fb5e6087eb1b2dcc20ad4d76fe97759aa27a0c99bff67106f4922f45568161a8cdf4ad2299f6d23
7,nomp_27000,8f14e45fceea167a5a36dedd4bea2543aab3238922bcc25a6f606eb525ffdc563c59dc048e8850243be8079a5c74d079
8,nomp_19390,c9f0f895fb98ab9159f51fd0297e236dc74d97b01eae257e44aa9d5bade97baf1ff1de774005f8da13f42943881c655f
9,nomp_32878,45c48cce2e2d7fbdea1afc51c7c6ad266f4922f45568161a8cdf4ad2299f6d2302e74f10e0327ad868d138f2b4fdd6f0


In [77]:
%%sql
EXPLAIN (analyse,buffers) 
SELECT * FROM produits 

Running query in 'pg'

4 rows affected.

QUERY PLAN
Seq Scan on produits (cost=0.00..1363.00 rows=50000 width=111) (actual time=0.006..1.838 rows=50000 loops=1)
Buffers: shared hit=863
Planning Time: 0.023 ms
Execution Time: 2.914 ms


### Analyse des résultats :

1. **Coût total estimé :**
   - Le coût total estimé par PostgreSQL est **1363.00**.
   - Le coût initial est **0.00**, ce qui signifie qu'il commence sans coût préalable.

2. **Temps réel d'exécution :**
   - Le temps réel d'exécution total est **9.340 ms**.
   - Le temps de parcours des tuples est entre **0.012 ms** et **5.747 ms**.

3. **Nombre de tuples estimés et taille :**
   - Nombre estimé de tuples en sortie : **50,000 rows**.
   - Taille moyenne estimée de chaque tuple : **111 bytes**.

4. **Nombre de blocs lus :**
   - **Buffers shared hit** : **863**, ce qui signifie que les blocs nécessaires étaient déjà en cache (shared hit) et aucun accès disque supplémentaire n'a été requis (shared read est absent).

5. **Chemins d'accès utilisés :**
   - PostgreSQL a choisi un **Seq Scan (Sequential Scan)** sur la table `produits`.
   - Cela implique que la table entière a été scannée ligne par ligne.

6. **Algorithme utilisé :**
   - **Sequential Scan** : Cet algorithme lit chaque ligne de la table sans utiliser d'index.

7. **Nœud le plus coûteux :**
   - Le nœud le plus coûteux dans ce plan est le **Seq Scan** lui-même, car il constitue l'unique nœud d'accès à la table `produits`.

8. **Interprétation :**
   - Le plan montre que PostgreSQL a scanné séquentiellement toute la table `produits`. Cela est probablement dû à l'absence d'un filtre ou d'une clause conditionnelle dans la requête (`WHERE`), ce qui rend un scan séquentiel optimal dans ce cas.
   - La faible taille des buffers partagés nécessaires (863) indique une bonne utilisation du cache.

### Recommandations :
- Si des filtres spécifiques sont ajoutés dans une future requête (par exemple, `WHERE`), créer un index sur les colonnes concernées pourrait améliorer les performances.
- Pour les requêtes listant toute la table, le **Sequential Scan** est généralement optimal et ne nécessite pas d'optimisation supplémentaire.

In [78]:
%%sql
EXPLAIN (analyse,buffers) 
SELECT nump, nomp FROM produits

Running query in 'pg'

4 rows affected.

QUERY PLAN
Seq Scan on produits (cost=0.00..1363.00 rows=50000 width=14) (actual time=0.008..2.508 rows=50000 loops=1)
Buffers: shared hit=863
Planning Time: 0.030 ms
Execution Time: 3.612 ms


In [79]:
%%sql
EXPLAIN (analyse,buffers) 
SELECT DISTINCT nump, nomp FROM produits

Running query in 'pg'

10 rows affected.

QUERY PLAN
HashAggregate (cost=1613.00..2113.00 rows=50000 width=14) (actual time=8.842..12.182 rows=50000 loops=1)
"Group Key: nump, nomp"
Batches: 1 Memory Usage: 3601kB
Buffers: shared hit=863
-> Seq Scan on produits (cost=0.00..1363.00 rows=50000 width=14) (actual time=0.005..2.172 rows=50000 loops=1)
Buffers: shared hit=863
Planning:
Buffers: shared hit=13
Planning Time: 0.069 ms
Execution Time: 13.276 ms


### Analyse des résultats pour ce plan d'exécution :

#### Structure du plan d'exécution :
1. **Node principal : HashAggregate**
   - **Coût estimé** : **1613.00..2113.00**
   - **Coût initial (cost=1613.00)** : Inclut le coût cumulé de toutes les opérations précédentes.
   - **Nombre estimé de lignes en sortie** : **50,000 rows**.
   - **Taille estimée de chaque tuple (width)** : **14 bytes**.
   - **Temps réel d'exécution** : **23.846..31.456 ms**.
   - **Memory Usage** : Utilisation de mémoire pour l’agrégation : **3601kB**.
   - **Group Key** : L'agrégation a été effectuée en groupant les colonnes `nump` et `nomp`.
   - **Batches** : Seulement 1 batch utilisé, ce qui signifie que tout tenait en mémoire.

2. **Sous-opération : Seq Scan**
   - PostgreSQL a utilisé un **Seq Scan** (Sequential Scan) sur la table `produits`.
   - **Coût estimé** : **0.00..1363.00**.
   - **Temps réel d'exécution** : **0.008..5.390 ms**.
   - **Nombre de tuples générés** : **50,000 rows**.
   - **Nombre de blocs lus** : **Buffers shared hit=863**, donc tout était en cache.

3. **Temps total d'exécution :**
   - Planification : **0.110 ms**.
   - Exécution : **33.356 ms**.

---

#### Observations détaillées :

1. **Coût estimé et temps réel :**
   - Le coût estimé du plan d'exécution est cohérent avec le temps réel.
   - Le temps réel d'exécution total est légèrement plus élevé (**33.356 ms**) à cause du temps nécessaire pour effectuer l'opération `HashAggregate`.

2. **Algorithmes utilisés :**
   - **HashAggregate** : Utilisé pour grouper les données sur les colonnes `nump` et `nomp`.
   - **Seq Scan** : Utilisé pour lire toute la table `produits`.

3. **Utilisation de mémoire et buffers :**
   - L'opération d'agrégation a utilisé **3601kB** de mémoire, ce qui est raisonnable.
   - Les buffers utilisés (**shared hit=863**) montrent que les données étaient déjà dans le cache et qu'aucun accès disque n'était nécessaire.

4. **Nœud le plus coûteux :**
   - Le nœud **HashAggregate** est clairement le plus coûteux en termes de temps d'exécution, car il effectue une opération lourde sur tous les tuples scannés.

5. **Effet du Seq Scan :**
   - Bien que le scan séquentiel soit rapide (5.390 ms), la partie lourde vient de l'agrégation.

---

#### Recommandations :
- Si cette requête est fréquente et les colonnes `nump` et `nomp` sont souvent utilisées pour des opérations de regroupement, créer un index ou une table matérialisée pourrait améliorer les performances.
- Pour un très grand volume de données dépassant la mémoire disponible, il serait nécessaire d'ajuster les paramètres de mémoire pour les opérations d'agrégation, ou envisager des partitions. 

Dans ce cas, la performance reste acceptable, et l’agrégation en mémoire sans batch supplémentaire montre une configuration adéquate pour le volume traité.

![alt text](pl.png "Plan")

In [80]:
%%sql
EXPLAIN (analyse,buffers) 
SELECT DISTINCT nump, nomp FROM produits ORDER BY nump

Running query in 'pg'

11 rows affected.

QUERY PLAN
Unique (cost=0.34..4672.29 rows=50000 width=14) (actual time=0.026..13.671 rows=50000 loops=1)
Buffers: shared hit=1001
-> Incremental Sort (cost=0.34..4422.29 rows=50000 width=14) (actual time=0.025..9.080 rows=50000 loops=1)
"Sort Key: nump, nomp"
Presorted Key: nump
Full-sort Groups: 1563 Sort Method: quicksort Average Memory: 26kB Peak Memory: 26kB
Buffers: shared hit=1001
-> Index Scan using produits_pkey on produits (cost=0.29..2172.29 rows=50000 width=14) (actual time=0.013..4.578 rows=50000 loops=1)
Buffers: shared hit=1001
Planning Time: 0.043 ms


### Analyse des résultats pour ce plan d'exécution :

---

#### Structure du plan d'exécution :
1. **Nœud principal : Unique**
   - **Coût estimé :** **0.34..4672.29**
   - **Temps réel d'exécution :** **0.038..24.861 ms**
   - **Nombre estimé de lignes en sortie :** **50,000 rows**
   - Le nœud `Unique` s'assure que les résultats finaux ne contiennent que des lignes distinctes sur la base des colonnes `nump` et `nomp`.

2. **Sous-opération : Incremental Sort**
   - **Coût estimé :** **0.34..4422.29**
   - **Temps réel d'exécution :** **0.037..16.762 ms**
   - **Sort Key :** `nump, nomp` (tri final sur ces colonnes).
   - **Presorted Key :** `nump` (les données sont partiellement triées par l'index utilisé).
   - **Sort Method :** `quicksort` avec une mémoire moyenne de **26kB**.
   - **Nombre de groupes triés (Full-sort Groups) :** **1563**, indiquant que la majeure partie des données étaient déjà triées par l'index.

3. **Sous-opération : Index Scan**
   - **Index utilisé :** `produits_pkey` (probablement un index sur la clé primaire `nump`).
   - **Coût estimé :** **0.29..2172.29**
   - **Temps réel d'exécution :** **0.020..9.053 ms**
   - **Nombre de lignes générées :** **50,000 rows**
   - **Buffers utilisés :** **shared hit=1001**, indiquant que toutes les données nécessaires étaient déjà en cache.

4. **Temps total d'exécution :**
   - **Planning Time :** **0.128 ms**
   - **Execution Time :** **26.775 ms**

---

#### Observations détaillées :

1. **Coût estimé et temps réel :**
   - Le coût total estimé est **4672.29**, mais le temps réel total est de **26.775 ms**, ce qui indique une efficacité du cache (toutes les données étaient accessibles en mémoire).

2. **Algorithmes utilisés :**
   - **Index Scan** : Accès direct aux données via un index (rapide).
   - **Incremental Sort** : PostgreSQL optimise le tri en tenant compte des données déjà triées partiellement grâce à l'index.
   - **Unique** : Supprime les doublons sur les colonnes `nump` et `nomp`.

3. **Utilisation de mémoire et buffers :**
   - Le tri incrémental utilise très peu de mémoire (26kB), ce qui est efficace.
   - Tous les **1001 buffers** nécessaires étaient dans le cache partagé (**shared hit**), donc pas d'accès disque.

4. **Nœud le plus coûteux :**
   - **Incremental Sort** est le nœud le plus coûteux dans ce plan d'exécution, bien qu'il soit optimisé par l'utilisation de l'index.

5. **Effet de l'index :**
   - L'utilisation de l'index `produits_pkey` réduit considérablement le coût initial de lecture des données, facilitant le tri partiel (`Presorted Key : nump`).

---

#### Recommandations :

1. **Performance :**
   - Le plan est performant grâce à l'utilisation de l'index et de l'algorithme de tri incrémental.
   - Pour des volumes de données plus importants, il pourrait être utile d'augmenter la mémoire allouée pour le tri dans PostgreSQL (`work_mem`) afin de réduire le risque de tri sur disque.

2. **Améliorations potentielles :**
   - Si cette requête est fréquente, un index sur les colonnes `nump, nomp` (par exemple, un **index composite**) pourrait améliorer davantage les performances, en évitant complètement l'opération de tri.

En l'état, ce plan d'exécution est bien optimisé et tire parti des ressources disponibles (cache et index). La faible utilisation de mémoire pour le tri indique un bon paramétrage de PostgreSQL.

![alt text](pl2.png "Plan")

In [81]:
%%sql
EXPLAIN (analyse,buffers) 
SELECT * FROM produits WHERE nomp='nomp_327'

Running query in 'pg'

6 rows affected.

QUERY PLAN
Seq Scan on produits (cost=0.00..1488.00 rows=2 width=111) (actual time=0.266..1.717 rows=2 loops=1)
Filter: ((nomp)::text = 'nomp_327'::text)
Rows Removed by Filter: 49998
Buffers: shared hit=863
Planning Time: 0.043 ms
Execution Time: 1.724 ms


### Analyse des résultats pour ce plan d'exécution :

---

#### Structure du plan d'exécution :
1. **Nœud principal : Seq Scan (Sequential Scan)**
   - **Coût estimé :** **0.00..1488.00**
   - **Temps réel d'exécution :** **0.713..2.166 ms**
   - **Nombre estimé de lignes en sortie :** **2 rows**
   - **Nombre réel de lignes générées :** **1 row**
   - **Filtre appliqué :** `(nomp)::text = 'nomp_327'::text`
   - **Lignes exclues par le filtre :** **49,999 rows**
   - **Buffers utilisés :** **shared hit=863**, indiquant que toutes les données nécessaires étaient déjà en cache.

2. **Temps total d'exécution :**
   - **Planning Time :** **0.041 ms**
   - **Execution Time :** **2.175 ms**

---

#### Observations détaillées :

1. **Coût estimé et temps réel :**
   - Le coût estimé par PostgreSQL est **1488.00**, mais le temps réel est **2.175 ms**, ce qui reflète une lecture rapide grâce à la présence des données dans le cache partagé.

2. **Filtrage des données :**
   - La condition `(nomp)::text = 'nomp_327'::text` a été appliquée en filtrant les lignes pendant le scan séquentiel.
   - Sur les **50,000 lignes**, **49,999** ont été ignorées (Rows Removed by Filter).

3. **Algorithmes utilisés :**
   - PostgreSQL a utilisé un **Seq Scan**, ce qui signifie que toute la table a été scannée ligne par ligne pour appliquer le filtre.

4. **Utilisation des buffers :**
   - Les **863 buffers** utilisés étaient entièrement en cache (**shared hit**), donc aucun accès disque supplémentaire n'a été nécessaire.

5. **Efficacité du Seq Scan :**
   - Bien que le scan séquentiel ait trouvé une seule ligne, il a dû parcourir les **50,000 lignes** de la table, ce qui est inefficace pour une recherche aussi spécifique.

6. **Estimation vs Réalité :**
   - PostgreSQL avait estimé que **2 lignes** seraient retournées, mais la requête n'en a trouvé qu'**1**. Cette légère différence pourrait être due à des statistiques obsolètes ou imprécises pour la colonne `nomp`.

---

#### Recommandations :

1. **Amélioration des performances :**
   - Créer un index sur la colonne `nomp` pourrait accélérer cette requête en évitant le scan séquentiel :
     ```sql
     CREATE INDEX idx_produits_nomp ON produits(nomp);
     ```
   - Avec cet index, PostgreSQL pourrait utiliser un **Index Scan** ou un **Bitmap Index Scan** pour rechercher directement les lignes correspondant au filtre.

2. **Actualisation des statistiques :**
   - Si les estimations sont régulièrement imprécises, mettre à jour les statistiques de la table pourrait améliorer les performances des requêtes :
     ```sql
     ANALYZE optimisation.produits;
     ```

3. **Alternative :**
   - Si des recherches similaires sont fréquentes, il pourrait être judicieux d'utiliser un index textuel pour des colonnes de type texte, surtout si elles sont sujettes à des recherches exactes ou partielles.

---

#### Conclusion :
Le plan actuel est sous-optimal pour cette requête, car il scanne toute la table pour trouver une seule ligne. Un index sur la colonne `nomp` améliorerait considérablement les performances et éviterait le scan séquentiel. Toutefois, le temps d'exécution reste acceptable grâce à l'utilisation du cache.

![alt text](pl3.png "Plan")

In [82]:
%%sql
EXPLAIN (analyse,buffers) 
SELECT numc, COUNT(*) AS nombre_commandes
FROM commandes
GROUP BY numc;

Running query in 'pg'

10 rows affected.

QUERY PLAN
HashAggregate (cost=6445.49..6863.85 rows=41836 width=12) (actual time=32.651..35.388 rows=49126 loops=1)
Group Key: numc
Batches: 1 Memory Usage: 4625kB
Buffers: shared hit=3447
-> Seq Scan on commandes (cost=0.00..5445.99 rows=199899 width=4) (actual time=0.004..7.580 rows=199899 loops=1)
Buffers: shared hit=3447
Planning:
Buffers: shared hit=29
Planning Time: 0.097 ms
Execution Time: 36.446 ms


### Analyse du plan d'exécution :

---

#### Structure du plan :

1. **Nœud principal : HashAggregate**
   - **Coût estimé :** **6445.30..6869.36**
   - **Temps réel d'exécution :** **78.207..81.435 ms**
   - **Nombre estimé de lignes en sortie :** **42,406 rows**
   - **Nombre réel de lignes en sortie :** **49,081 rows**
   - **Mémoire utilisée :** **4625kB**, tout le traitement a été réalisé en mémoire (1 seul batch).
   - **Clé de regroupement :** `numc`.

2. **Sous-opération : Seq Scan**
   - **Table scannée :** `commandes`
   - **Coût estimé :** **0.00..5445.87**
   - **Temps réel d'exécution :** **0.009..13.626 ms**
   - **Nombre de lignes générées :** **199,887 rows**
   - **Buffers utilisés :** **shared hit=3447**, ce qui indique que toutes les données nécessaires étaient dans le cache.

3. **Temps total d'exécution :**
   - **Planning Time :** **0.160 ms**
   - **Execution Time :** **82.612 ms**

---

#### Observations détaillées :

1. **Coût estimé vs Temps réel :**
   - Le coût estimé par PostgreSQL pour la requête est cohérent avec le temps réel observé. Le traitement principal est l'agrégation (HashAggregate).

2. **HashAggregate :**
   - Le regroupement (`GROUP BY numc`) est réalisé avec un **HashAggregate**, qui est adapté dans ce cas puisque les données tiennent en mémoire (batch unique).
   - La mémoire utilisée (**4625kB**) reste raisonnable pour traiter environ **199,887 lignes**.

3. **Seq Scan :**
   - La table `commandes` est lue entièrement via un **Seq Scan**.
   - Cette méthode est utilisée car il n'y a pas d'index spécifique sur `numc` qui permettrait d'accélérer cette opération.

4. **Utilisation des buffers :**
   - Tous les **3447 buffers** nécessaires étaient déjà en cache partagé (**shared hit**), donc aucun accès disque n'a été nécessaire, ce qui améliore le temps d'exécution.

5. **Estimation vs Réalité :**
   - PostgreSQL a estimé que **42,406** groupes seraient créés, mais le résultat réel est de **49,081 groupes**. Cette différence pourrait refléter une légère imprécision dans les statistiques de la colonne `numc`.

---

#### Recommandations pour amélioration :

1. **Création d’un index sur `numc` :**
   - Si cette requête ou des requêtes similaires sont fréquentes, un index sur `numc` pourrait accélérer l’opération :
     ```sql
     CREATE INDEX idx_commandes_numc ON commandes(numc);
     ```

2. **Mise à jour des statistiques :**
   - Les statistiques de la colonne `numc` pourraient être mises à jour pour affiner les estimations :
     ```sql
     ANALYZE optimisation.commandes;
     ```

3. **Partitionnement éventuel :**
   - Si la table `commandes` devient beaucoup plus volumineuse, envisagez un partitionnement basé sur `numc` pour réduire le coût des scans.

---

#### Conclusion :
Le plan actuel est performant grâce à l’utilisation du cache, mais un **Seq Scan** pour une opération d’agrégation sur une colonne spécifique (`numc`) peut être optimisé avec un index. Le temps d’exécution total est acceptable pour cette taille de données.


![alt text](pl4.png "Plan")

In [83]:
%%sql
-- Variante 1 : Utilisation de la syntaxe explicite JOIN
EXPLAIN (analyse,buffers) 
SELECT cl.nomc, co.datecom
FROM optimisation.clients cl
JOIN optimisation.commandes co
ON cl.numc = co.numc;


Running query in 'pg'

14 rows affected.

QUERY PLAN
Hash Join (cost=1988.00..7958.77 rows=199899 width=14) (actual time=5.395..35.894 rows=199899 loops=1)
Hash Cond: (co.numc = cl.numc)
Buffers: shared hit=4310
-> Seq Scan on commandes co (cost=0.00..5445.99 rows=199899 width=8) (actual time=0.006..10.865 rows=199899 loops=1)
Buffers: shared hit=3447
-> Hash (cost=1363.00..1363.00 rows=50000 width=14) (actual time=5.380..5.381 rows=50000 loops=1)
Buckets: 65536 Batches: 1 Memory Usage: 2856kB
Buffers: shared hit=863
-> Seq Scan on clients cl (cost=0.00..1363.00 rows=50000 width=14) (actual time=0.003..2.390 rows=50000 loops=1)
Buffers: shared hit=863


In [84]:
%%sql
-- Variante 2 : Utilisation de la syntaxe implicite JOIN
EXPLAIN (analyse,buffers) 
SELECT cl.nomc, co.datecom
FROM optimisation.clients cl, optimisation.commandes co
WHERE cl.numc = co.numc;

Running query in 'pg'

14 rows affected.

QUERY PLAN
Hash Join (cost=1988.00..7958.77 rows=199899 width=14) (actual time=5.343..34.321 rows=199899 loops=1)
Hash Cond: (co.numc = cl.numc)
Buffers: shared hit=4310
-> Seq Scan on commandes co (cost=0.00..5445.99 rows=199899 width=8) (actual time=0.005..9.321 rows=199899 loops=1)
Buffers: shared hit=3447
-> Hash (cost=1363.00..1363.00 rows=50000 width=14) (actual time=5.329..5.330 rows=50000 loops=1)
Buckets: 65536 Batches: 1 Memory Usage: 2856kB
Buffers: shared hit=863
-> Seq Scan on clients cl (cost=0.00..1363.00 rows=50000 width=14) (actual time=0.003..2.403 rows=50000 loops=1)
Buffers: shared hit=863


In [85]:
%%sql
explain -- Attention, pas de mot clé "analyse" ici car la requête ne termine pas !
select distinct nomc, nomp
from clients, produits, concerne, commandes, livraisons
where	clients.numC = commandes.numc
	and commandes.datecom = concerne.datecom
	and commandes.numc=concerne.numc
	and produits.nump=concerne.nump

Running query in 'pg'

24 rows affected.

QUERY PLAN
Unique (cost=46368.40..1543341524.16 rows=675217725 width=20)
-> Nested Loop (cost=46368.40..1102420785.18 rows=88184147796 width=20)
-> Gather Merge (cost=46368.40..116160.24 rows=599244 width=20)
Workers Planned: 2
-> Sort (cost=45368.38..45992.59 rows=249685 width=20)
"Sort Key: clients.nomc, produits.nomp"
-> Parallel Hash Join (cost=9505.28..17863.93 rows=249685 width=20)
Hash Cond: ((clients.numc = commandes.numc) AND (concerne.datecom = commandes.datecom))
-> Hash Join (cost=3976.00..11023.81 rows=249685 width=32)
Hash Cond: (concerne.nump = produits.nump)


![alt text](pl5.png "Plan")

Requête réécrite pour optimisation
Pour améliorer la requête, plusieurs actions peuvent être prises :
Utiliser la syntaxe explicite JOIN pour réduire le risque d’un produit cartésien.
Éliminer les combinaisons inutiles en ne joignant que les tables réellement nécessaires.
Ajouter des index sur les colonnes de jointure si cela n’a pas encore été fait.
Retarder l’application de DISTINCT en réduisant au maximum les données intermédiaires.

In [87]:
%%sql
explain -- Attention, pas de mot clé "analyse" ici car la requête ne termine pas !
SELECT DISTINCT cl.nomc, pr.nomp
FROM optimisation.clients cl
JOIN optimisation.commandes co ON cl.numc = co.numc
JOIN optimisation.concerne cn ON co.numc = cn.numc AND co.datecom = cn.datecom
JOIN optimisation.produits pr ON cn.nump = pr.nump;

Running query in 'pg'

16 rows affected.

QUERY PLAN
HashAggregate (cost=78429.91..91444.74 rows=599244 width=20)
"Group Key: cl.nomc, pr.nomp"
Planned Partitions: 16
-> Hash Join (cost=13201.48..35359.24 rows=599244 width=20)
Hash Cond: (cn.nump = pr.nump)
-> Hash Join (cost=11213.48..31798.10 rows=599244 width=14)
Hash Cond: (co.numc = cl.numc)
-> Hash Join (cost=9225.48..28236.95 rows=599244 width=12)
Hash Cond: ((cn.numc = co.numc) AND (cn.datecom = co.datecom))
-> Seq Scan on concerne cn (cost=0.00..9232.44 rows=599244 width=12)


![alt text](pl6.png "Plan")

In [88]:
%config SqlMagic.autocommit = False #commit automatique après chaque requête

In [119]:
%%sql
Explain analyse
Select *
from commandes
where	numC='109000' and datecom='2020-03-31';

Running query in 'pg'

4 rows affected.

QUERY PLAN
Index Scan using commandes_pkey on commandes (cost=0.42..8.44 rows=1 width=105) (actual time=0.020..0.020 rows=0 loops=1)
Index Cond: ((datecom = '2020-03-31'::date) AND (numc = 109000))
Planning Time: 0.183 ms
Execution Time: 0.039 ms


In [117]:
%%sql

begin;
  CREATE UNIQUE INDEX idx_clients_numc ON commandes(numc,datecom);

Explain analyse
Select *
from commandes
where	numC='109000' and datecom='2020-03-31';





Running query in 'pg'

4 rows affected.

QUERY PLAN
Index Scan using idx_clients_numc on commandes (cost=0.42..8.44 rows=1 width=105) (actual time=0.013..0.013 rows=0 loops=1)
Index Cond: ((numc = 109000) AND (datecom = '2020-03-31'::date))
Planning Time: 0.112 ms
Execution Time: 0.022 ms


In [118]:
%%sql
rollback;

Running query in 'pg'

++
||
++
++

In [123]:
%%sql
Explain analyse
Select *
from commandes
where numC='109000';

Running query in 'pg'

4 rows affected.

QUERY PLAN
Index Scan using commandes_pkey on commandes (cost=0.42..4131.71 rows=5 width=105) (actual time=8.514..8.516 rows=0 loops=1)
Index Cond: (numc = 109000)
Planning Time: 0.113 ms
Execution Time: 8.536 ms


In [124]:
%%sql

begin;
  CREATE UNIQUE INDEX idx_clients_numc ON commandes(numc,datecom);

Explain analyse
Select *
from commandes
where numC='109000';





Running query in 'pg'

6 rows affected.

QUERY PLAN
Bitmap Heap Scan on commandes (cost=4.46..23.95 rows=5 width=105) (actual time=0.015..0.016 rows=0 loops=1)
Recheck Cond: (numc = 109000)
-> Bitmap Index Scan on idx_clients_numc (cost=0.00..4.46 rows=5 width=0) (actual time=0.013..0.013 rows=0 loops=1)
Index Cond: (numc = 109000)
Planning Time: 0.101 ms
Execution Time: 0.025 ms


In [125]:
%%sql
rollback;

Running query in 'pg'

++
||
++
++

In [126]:
%%sql

begin;
  CREATE INDEX idx_concerne_nump_numc ON optimisation.concerne(nump, numc);
  CREATE INDEX idx_clients_nomc ON optimisation.clients(nomc);


EXPLAIN ANALYZE
SELECT nomp
FROM optimisation.produits p
JOIN optimisation.concerne co USING(nump)
JOIN optimisation.clients c USING(numc)
WHERE c.nomc = 'nomc_1287';


Running query in 'pg'

18 rows affected.

QUERY PLAN
Gather (cost=1012.36..7409.94 rows=24 width=10) (actual time=5.475..20.692 rows=29 loops=1)
Workers Planned: 2
Workers Launched: 2
-> Nested Loop (cost=12.36..6407.54 rows=10 width=10) (actual time=2.845..13.627 rows=10 loops=3)
-> Hash Join (cost=12.07..6404.40 rows=10 width=4) (actual time=2.835..13.590 rows=10 loops=3)
Hash Cond: (co.numc = c.numc)
-> Parallel Seq Scan on concerne co (cost=0.00..5736.85 rows=249685 width=8) (actual time=0.007..6.303 rows=199748 loops=3)
-> Hash (cost=12.04..12.04 rows=2 width=4) (actual time=0.024..0.025 rows=2 loops=3)
Buckets: 1024 Batches: 1 Memory Usage: 9kB
-> Bitmap Heap Scan on clients c (cost=4.31..12.04 rows=2 width=4) (actual time=0.018..0.022 rows=2 loops=3)


In [127]:
%%sql
rollback;

Running query in 'pg'

++
||
++
++

In [128]:
%%sql
EXPLAIN (analyse,buffers) 
Select numc
  from clients 
  where numc not in (select numc from livraisons); 

Running query in 'pg'

12 rows affected.

QUERY PLAN
Index Only Scan using clients_pkey on clients (cost=2777.78..4208.78 rows=25000 width=4) (actual time=41.819..49.727 rows=9467 loops=1)
Filter: (NOT (ANY (numc = (hashed SubPlan 1).col1)))
Rows Removed by Filter: 40533
Heap Fetches: 0
Buffers: shared hit=1077
SubPlan 1
-> Seq Scan on livraisons (cost=0.00..2409.59 rows=147159 width=4) (actual time=0.008..12.638 rows=147159 loops=1)
Buffers: shared hit=938
Planning:
Buffers: shared hit=13 dirtied=1


In [129]:
%%sql
EXPLAIN (analyse,buffers) 
 Select numc
  from clients 
      except
  select numc 
  from livraisons; 

Running query in 'pg'

15 rows affected.

QUERY PLAN
HashSetOp Except (cost=0.29..7166.16 rows=50000 width=8) (actual time=71.474..72.287 rows=9467 loops=1)
Buffers: shared hit=1077
-> Append (cost=0.29..6673.27 rows=197159 width=8) (actual time=0.030..44.094 rows=197159 loops=1)
Buffers: shared hit=1077
"-> Subquery Scan on ""*SELECT* 1"" (cost=0.29..1806.29 rows=50000 width=8) (actual time=0.030..11.830 rows=50000 loops=1)"
Buffers: shared hit=139
-> Index Only Scan using clients_pkey on clients (cost=0.29..1306.29 rows=50000 width=4) (actual time=0.028..5.811 rows=50000 loops=1)
Heap Fetches: 0
Buffers: shared hit=139
"-> Subquery Scan on ""*SELECT* 2"" (cost=0.00..3881.18 rows=147159 width=8) (actual time=0.020..19.730 rows=147159 loops=1)"


In [130]:
%%sql
EXPLAIN (analyse,buffers) 
Select numc
  from clients 
  where not exists (select numc from livraisons
                    where livraisons.numc=clients.numc);

Running query in 'pg'

15 rows affected.

QUERY PLAN
Hash Right Anti Join (cost=1931.29..5796.85 rows=17331 width=4) (actual time=55.259..56.610 rows=9467 loops=1)
Hash Cond: (livraisons.numc = clients.numc)
Buffers: shared hit=1077
-> Seq Scan on livraisons (cost=0.00..2409.59 rows=147159 width=4) (actual time=0.034..10.734 rows=147159 loops=1)
Buffers: shared hit=938
-> Hash (cost=1306.29..1306.29 rows=50000 width=4) (actual time=20.770..20.771 rows=50000 loops=1)
Buckets: 65536 Batches: 1 Memory Usage: 2270kB
Buffers: shared hit=139
-> Index Only Scan using clients_pkey on clients (cost=0.29..1306.29 rows=50000 width=4) (actual time=0.014..8.773 rows=50000 loops=1)
Heap Fetches: 0


![alt text](pl11.png "Plan")
![alt text](pl12.png "Plan")
![alt text](pl13.png "Plan")

In [131]:
%%sql

begin;
  CREATE INDEX idx_clients_upper_nomc ON clients (UPPER(nomc));



EXPLAIN ANALYZE
SELECT UPPER(nomc), adressec
FROM clients
WHERE UPPER(nomc) = 'NOMC_1725';



Running query in 'pg'

7 rows affected.

QUERY PLAN
Bitmap Heap Scan on clients (cost=6.23..555.64 rows=250 width=129) (actual time=0.019..0.019 rows=1 loops=1)
Recheck Cond: (upper((nomc)::text) = 'NOMC_1725'::text)
Heap Blocks: exact=1
-> Bitmap Index Scan on idx_clients_upper_nomc (cost=0.00..6.17 rows=250 width=0) (actual time=0.009..0.009 rows=1 loops=1)
Index Cond: (upper((nomc)::text) = 'NOMC_1725'::text)
Planning Time: 0.093 ms
Execution Time: 0.028 ms


In [132]:
%%sql
rollback;

Running query in 'pg'

++
||
++
++

In [133]:
%%sql
begin;
  explain
  delete from commandes
  where datecom in (  select datecom	
  				            from commandes
  			            	order by datecom
  			            	limit 200);
  

Running query in 'pg'

8 rows affected.

QUERY PLAN
Delete on commandes (cost=10.53..6103.60 rows=0 width=0)
-> Hash Semi Join (cost=10.53..6103.60 rows=10997 width=34)
"Hash Cond: (commandes.datecom = ""ANY_subquery"".datecom)"
-> Seq Scan on commandes (cost=0.00..5445.99 rows=199899 width=10)
-> Hash (cost=8.03..8.03 rows=200 width=32)
"-> Subquery Scan on ""ANY_subquery"" (cost=0.42..8.03 rows=200 width=32)"
-> Limit (cost=0.42..6.03 rows=200 width=4)
-> Index Only Scan using commandes_pkey on commandes commandes_1 (cost=0.42..5610.90 rows=199899 width=4)


In [134]:
%%sql
rollback;

Running query in 'pg'

++
||
++
++

Explications des résultats
Temps consacré à la sous-requête (SELECT) :
Le tri de la table entière sur datecom est coûteux si la table est grande et qu’il n’y a pas d’index sur cette colonne. PostgreSQL pourrait effectuer un Seq Scan suivi d’un tri.
Temps consacré à la suppression :
PostgreSQL doit rechercher chaque ligne correspondant aux 200 dates extraites. Si un index n’existe pas sur datecom, cela implique un scan séquentiel pour chaque suppression.
Coût global :
L’absence d’index sur datecom rend l’opération plus lente, car PostgreSQL effectue des recherches répétées sans optimisation.

In [ ]:
%%sql
EXPLAIN (analyse,buffers) 
SELECT DISTINCT nump, nomp FROM produits

In [135]:
%%sql
CREATE INDEX idx_commandes_datecom ON commandes(datecom);
begin;
  explain
  delete from commandes
  where datecom in (  select datecom	
  				            from commandes
  			            	order by datecom
  			            	limit 200);
  

Running query in 'pg'

8 rows affected.

QUERY PLAN
Delete on commandes (cost=8.49..6101.56 rows=0 width=0)
-> Hash Semi Join (cost=8.49..6101.56 rows=10997 width=34)
"Hash Cond: (commandes.datecom = ""ANY_subquery"".datecom)"
-> Seq Scan on commandes (cost=0.00..5445.99 rows=199899 width=10)
-> Hash (cost=5.99..5.99 rows=200 width=32)
"-> Subquery Scan on ""ANY_subquery"" (cost=0.29..5.99 rows=200 width=32)"
-> Limit (cost=0.29..3.99 rows=200 width=4)
-> Index Only Scan using idx_commandes_datecom on commandes commandes_1 (cost=0.29..3694.78 rows=199899 width=4)


In [136]:
%%sql
rollback;

Running query in 'pg'

++
||
++
++

In [139]:
%%sql
CREATE SCHEMA optimisationpartition;

CREATE TABLE optimisationpartition.concerne (
    numc INT,
    datecom DATE,
    nump INT,
    PRIMARY KEY (numc, datecom, nump)
) PARTITION BY RANGE (datecom);

CREATE TABLE optimisationpartition.concerne_old PARTITION OF optimisationpartition.concerne
FOR VALUES FROM (MINVALUE) TO (CURRENT_DATE - INTERVAL '3 years');



CREATE TABLE optimisationpartition.concerne_recent PARTITION OF optimisationpartition.concerne
FOR VALUES FROM (CURRENT_DATE - INTERVAL '3 years') TO (CURRENT_DATE - INTERVAL '1 year');


CREATE TABLE optimisationpartition.concerne_current PARTITION OF optimisationpartition.concerne
FOR VALUES FROM (CURRENT_DATE - INTERVAL '1 year') TO (MAXVALUE);




Running query in 'pg'

++
||
++
++

In [141]:
%%sql
INSERT INTO optimisationpartition.concerne (numc, datecom, nump)
SELECT numc, datecom, nump
FROM optimisation.concerne;

SELECT * FROM optimisationpartition.concerne_old;
SELECT * FROM optimisationpartition.concerne_recent;
SELECT * FROM optimisationpartition.concerne_current;


Running query in 'pg'

RuntimeError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "concerne_current_pkey"
DETAIL:  Key (numc, datecom, nump)=(8756, 2024-01-11, 648) already exists.

[SQL: INSERT INTO optimisationpartition.concerne (numc, datecom, nump)
SELECT numc, datecom, nump
FROM optimisation.concerne;]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
If you need help solving this issue, send us a message: https://ploomber.io/community


In [142]:
%%sql
EXPLAIN ANALYZE
SELECT * FROM optimisationpartition.concerne
WHERE datecom < CURRENT_DATE - INTERVAL '3 years';


Running query in 'pg'

6 rows affected.

QUERY PLAN
Append (cost=0.00..15821.75 rows=418995 width=12) (actual time=0.009..67.932 rows=419021 loops=1)
Subplans Removed: 2
-> Seq Scan on concerne_old concerne_1 (cost=0.00..9597.87 rows=418981 width=12) (actual time=0.008..48.165 rows=419021 loops=1)
Filter: (datecom < (CURRENT_DATE - '3 years'::interval))
Planning Time: 0.361 ms
Execution Time: 77.780 ms


In [143]:
%%sql
EXPLAIN ANALYZE
SELECT * FROM optimisationpartition.concerne
WHERE datecom >= CURRENT_DATE - INTERVAL '1 year';

Running query in 'pg'

6 rows affected.

QUERY PLAN
Append (cost=0.00..14027.26 rows=60097 width=12) (actual time=0.008..20.297 rows=60051 loops=1)
Subplans Removed: 2
-> Seq Scan on concerne_current concerne_1 (cost=0.00..1375.89 rows=60047 width=12) (actual time=0.007..14.882 rows=60051 loops=1)
Filter: (datecom >= (CURRENT_DATE - '1 year'::interval))
Planning Time: 0.122 ms
Execution Time: 22.767 ms
